# Hyperparameter tuning Neural network
**Keras tuner- Decide num of hidden layers and neuron in neural network**

In [ ]:
#pip install keras-tuner

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from keras_tuner.tuners import RandomSearch

from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

## Importing Dataset

In [2]:
mnist=tf.keras.datasets.mnist

In [15]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()

In [16]:
# Normalize
X_train,X_test=X_train/255.0 ,X_test/255.0

In [17]:
print("X_train Shape",X_train.shape)

X_train Shape (60000, 28, 28)


## Hyperparameter Tuning

**hp.Int-->** Incase we know inputs are integer

**hp.choice-->** Incase we have several choice among the given several parameters

In [10]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    model.add(layers.Dense(units=hp.Int('units_',
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

**max_trails=5**-- Deep neural network fits thousands  of permutation and combinantion of model to find best hyperparameter, using this keyword model onlytry 5 models

**executions_per_trial=3** -- In each try it will try with 3 diferent models.

In [11]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dict',
    project_name='Hypertuning')

INFO:tensorflow:Reloading Oracle from existing project my_dict\Hypertuning\oracle.json


**Check the hyperparamets**

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units_ (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


**Fitting the model**

In [13]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 6 Complete [00h 02m 02s]
val_accuracy: 0.9516999920209249

Best val_accuracy So Far: 0.9760333299636841
Total elapsed time: 00h 13m 55s
INFO:tensorflow:Oracle triggered exit


**Summary of the result**

In [14]:
tuner.results_summary()

Results summary
Results in my_dict\Hypertuning
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units_: 96
learning_rate: 0.001
Score: 0.9760333299636841
Trial summary
Hyperparameters:
units_: 512
learning_rate: 0.0001
Score: 0.9678333401679993
Trial summary
Hyperparameters:
units_: 256
learning_rate: 0.0001
Score: 0.9606333374977112
Trial summary
Hyperparameters:
units_: 128
learning_rate: 0.0001
Score: 0.9516999920209249
Trial summary
Hyperparameters:
units_: 96
learning_rate: 0.0001
Score: 0.945633331934611


# Using Loops

**Int('units_' + str(i)**--> we used this coz in case of epochs we can keep an eye on units assigned as in epochs it as units_1,units_2 and so on 

In [6]:
def build_model2(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    for i in range(hp.Int("layers",2,6)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=10,
                                            max_value=250,
                                            step=25),
                               activation=hp.Choice("act"+str(i),["LeakyReLU",'relu'])))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Note-
Always give separate name to `directory` and `project_name`,otherwise it won't replace the tuner it will only take previous defined tuner.

In [10]:
tuner_nw = RandomSearch(
    build_model2,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='my_dict_loop',
    project_name='Hypertuning_loop')

In [11]:
tuner_nw.search_space_summary()

Search space summary
Default search space size: 6
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 250, 'step': 25, 'sampling': None}
act0 (Choice)
{'default': 'LeakyReLU', 'conditions': [], 'values': ['LeakyReLU', 'relu'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 250, 'step': 25, 'sampling': None}
act1 (Choice)
{'default': 'LeakyReLU', 'conditions': [], 'values': ['LeakyReLU', 'relu'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [12]:
tuner_nw.search(X_train,y_train,epochs=3,validation_data=(X_test,y_test))

Trial 3 Complete [00h 01m 23s]
val_accuracy: 0.9300333460172018

Best val_accuracy So Far: 0.966533342997233
Total elapsed time: 00h 04m 46s
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner_nw.results_summary()

Results summary
Results in my_dict_loop\Hypertuning_loop
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
layers: 6
units_0: 235
act0: LeakyReLU
units_1: 110
act1: relu
learning_rate: 0.001
units_2: 210
act2: LeakyReLU
units_3: 185
act3: relu
units_4: 10
act4: LeakyReLU
units_5: 10
act5: LeakyReLU
Score: 0.966533342997233
Trial summary
Hyperparameters:
layers: 4
units_0: 60
act0: relu
units_1: 85
act1: LeakyReLU
learning_rate: 0.01
units_2: 10
act2: LeakyReLU
units_3: 10
act3: LeakyReLU
Score: 0.9568333427111307
Trial summary
Hyperparameters:
layers: 5
units_0: 35
act0: LeakyReLU
units_1: 85
act1: relu
learning_rate: 0.01
units_2: 85
act2: LeakyReLU
units_3: 160
act3: relu
units_4: 60
act4: relu
units_5: 235
act5: relu
Score: 0.9300333460172018


# Different Problem

## loading dataset

In [2]:
df=pd.read_csv('Real_Combine.csv')
df.drop("Unnamed: 0",axis=1,inplace=True)
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


**This dataset is regarding AQI where PMI 2.5 show the air quality index**

## Checking null

In [3]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64

### Removing null

In [4]:
df.dropna(axis=0,inplace=True)

In [5]:
df.isnull().sum()

T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    0
dtype: int64

#### Splitting the data

In [6]:
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df,train_size=0.8,random_state=100)

In [7]:
print(df_train.shape)
print(df_test.shape)

(873, 9)
(219, 9)


In [8]:
## scaling
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [9]:
cols=df_train.columns
cols

Index(['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'], dtype='object')

In [10]:
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=[cols])
df_train.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,-0.133179,-0.263944,-0.069670,0.310347,0.444521,0.110612,-0.794044,-0.630246,-0.718550
1,0.588877,0.367806,1.096328,-0.717711,1.267366,0.378845,-0.055615,0.124819,-0.966534
2,-1.257919,-0.895695,-1.289900,1.114341,0.444521,-1.230556,-1.532473,-1.119640,1.372811
3,1.047105,1.119890,0.472655,-0.559548,-2.277199,0.512962,0.814677,0.376508,0.029900
4,0.741620,0.593431,1.096328,-0.335484,-0.441621,-0.157622,1.315754,1.439192,-0.564060


In [11]:
df_test=pd.DataFrame(scaler.transform(df_test),columns=[cols])
df_test.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,-1.966089,-2.535237,-1.628853,1.430666,1.457254,-1.632907,-0.161105,-0.630246,0.005902
1,-0.646949,-0.444444,-0.923831,0.745294,-1.454354,0.781196,0.339972,0.698110,0.924840
2,-0.508092,-0.594861,-0.435739,0.863917,0.254633,-1.364673,-0.345712,-0.630246,1.322814
3,0.686077,0.397889,0.879398,-1.337182,1.077479,0.512962,0.128993,0.376508,-0.648055
4,0.699963,0.458056,0.933631,-1.258101,0.634408,-0.157622,1.104774,0.376508,-0.917037


In [12]:
## Find X_train and Y_train
X_train=df_train.iloc[:,0:-1]
y_train=df_train.iloc[:,-1]

In [13]:
y_train.head()

0   -0.718550
1   -0.966534
2    1.372811
3    0.029900
4   -0.564060
Name: (PM 2.5,), dtype: float64

In [14]:
## Find X_test and y_test
X_test=df_test.iloc[:,0:-1]
y_test=df_test.iloc[:,-1]

In [15]:
y_test.head()

0    0.005902
1    0.924840
2    1.322814
3   -0.648055
4   -0.917037
Name: (PM 2.5,), dtype: float64

## Hyperparameters
* How many number of hidden layers we should have?
* How many number of neurons we should have in hidden layers?
* Learning Rate

In [16]:
def final_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [17]:
final_tuner = RandomSearch(
    final_model,
    objective='val_mean_absolute_error',
    max_trials=3,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [18]:
final_tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [19]:
final_tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 12s]
val_mean_absolute_error: 0.47732870777448017

Best val_mean_absolute_error So Far: 0.4519602755705516
Total elapsed time: 00h 00m 48s
INFO:tensorflow:Oracle triggered exit


In [20]:
final_tuner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 9
units_0: 416
units_1: 288
learning_rate: 0.001
units_2: 448
units_3: 224
units_4: 448
units_5: 448
units_6: 448
units_7: 64
units_8: 128
Score: 0.4519602755705516
Trial summary
Hyperparameters:
num_layers: 5
units_0: 352
units_1: 128
learning_rate: 0.01
units_2: 64
units_3: 480
units_4: 480
units_5: 416
units_6: 416
units_7: 416
units_8: 352
Score: 0.47732870777448017
Trial summary
Hyperparameters:
num_layers: 9
units_0: 320
units_1: 96
learning_rate: 0.01
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 0.49050647020339966


## Summary
* Here best hyperparameter is having 9 layer since val_mean_absolute_error is 45% error, as we are taking the lowest error
* units give the num of neurons like-
* units_0: 416 neurons
* units_1: 288 neurons
* units_2: 448 neurons
* units_3: 224 neurons
* units_4: 448 neurons